# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go

# Rheological values

In [ ]:
def storage_modulus_at_eq(sweep_df):
    return sweep_df['Storage Modulus'].iloc[0]

def tau_f(sweep_df, skip_initial=10):
    # Returns the index and shear stress value of the first point where Loss Modulus and Storage Modulus intersect. Returns shear stress at 0 if they do not intersect.
    skipped_df = sweep_df.iloc[skip_initial:]
    larger_modulus = ''
    if skipped_df["Storage Modulus"].iloc[0] > skipped_df["Loss Modulus"].iloc[0]:
        larger_modulus = 'Storage Modulus'
        smaller_modulus = 'Loss Modulus'
    else:
        larger_modulus = 'Loss Modulus'
        smaller_modulus = 'Storage Modulus'

    idx = skipped_df[skipped_df[larger_modulus] < skipped_df[smaller_modulus]].index.tolist()
    if len(idx) == 0:
        return (0, sweep_df['Shear Stress'].iloc[0])
    else:
        return (idx[0]-1, sweep_df["Shear Stress"].iloc[idx[0]-1])

def tau_y_old(sweep_df, percent_drop=0.05, skip_initial=10):
    # Returns the shear stress at the point the storage modulus starts decreasing
    skipped_df = sweep_df.iloc[skip_initial:]

    max_storage_modulus = skipped_df["Storage Modulus"].iloc[0]

    threshold = max_storage_modulus * (1 - percent_drop)

    idx = skipped_df[skipped_df["Storage Modulus"] < threshold].index.tolist()
    print(idx)
    return (idx[0], sweep_df["Shear Stress"].iloc[idx[0]])


def tau_y(sweep_df, percent_drop=0.05, skip_initial=10):
    # Returns the shear stress at the point the storage modulus drops more than percent_drop from the previous index
    skipped_df = sweep_df.iloc[skip_initial:].reset_index(drop=True)

    # Iterate through the dataframe to find the first point where drop exceeds percent_drop
    for i in range(1, len(skipped_df)):
        previous_modulus = skipped_df["Storage Modulus"].iloc[i - 1]
        current_modulus = skipped_df["Storage Modulus"].iloc[i]

        # Calculate the percentage drop
        drop_percent = (previous_modulus - current_modulus) / previous_modulus

        if drop_percent > percent_drop:
            actual_idx = skipped_df.index[i] + skip_initial
            print([actual_idx])
            return (actual_idx, sweep_df["Shear Stress"].iloc[actual_idx])

    # If no drop is found, return the last index
    actual_idx = skipped_df.index[-1] + skip_initial
    print([actual_idx])
    return (actual_idx, sweep_df["Shear Stress"].iloc[actual_idx])

In [ ]:
def _prep_sweep(df):
    # map new column names to the older short names used by helper functions
    mapping = {
        'Strain [%]': 'Strain',
        'Shear Stress [Pa]': 'Shear Stress',
        'Storage Modulus [Pa]': 'Storage Modulus',
        'Loss Modulus [Pa]': 'Loss Modulus'
    }
    d = df.rename(columns=mapping).copy()
    for c in mapping.values():
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors='coerce')
    return d

def plot_rheometry(sweep_df, title="Rheometry Data"):
    d = _prep_sweep(sweep_df)

    plt.figure(figsize=(20, 12))

    mask_g = (d['Strain'] > 0) & (d['Storage Modulus'] > 0)
    plt.plot(d['Strain'][mask_g], d['Storage Modulus'][mask_g],
             label='Storage Modulus', marker='s', color='blue')

    mask_l = (d['Strain'] > 0) & (d['Loss Modulus'] > 0)
    plt.plot(d['Strain'][mask_l], d['Loss Modulus'][mask_l],
             label='Loss Modulus', marker='^', color='blue')

    # TAU_Y point
    try:
        tau_y_id, tau_y_val = tau_y(d, 0.05)
        plt.scatter(
            d["Strain"].iloc[tau_y_id],
            tau_y_val,
            label="tau_y",
            color="red",
            s=100
        )
    except Exception:
        pass

    plt.xlabel('Strain (%)')
    plt.ylabel('Pa')
    plt.title(title)
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_rheometry_plotly(sweep_df, title="Rheometry Data"):
    d = _prep_sweep(sweep_df)

    fig = go.Figure()

    mask_g = d['Strain'].notna() & d['Storage Modulus'].notna() & (d['Storage Modulus'] > 0)
    fig.add_trace(go.Scatter(
        x=d['Strain'][mask_g],
        y=d['Storage Modulus'][mask_g],
        mode='markers+lines',
        name='Storage Modulus'
    ))

    mask_l = d['Strain'].notna() & d['Loss Modulus'].notna() & (d['Loss Modulus'] > 0)
    fig.add_trace(go.Scatter(
        x=d['Strain'][mask_l],
        y=d['Loss Modulus'][mask_l],
        mode='markers+lines',
        name='Loss Modulus'
    ))

    # TAU_Y point
    try:
        tau_y_id, tau_y_val = tau_y(d, 0.02, 15)
        fig.add_trace(go.Scatter(
            x=[d["Strain"].iloc[tau_y_id]],
            y=[tau_y_val],
            mode='markers',
            name='tau_y',
            marker=dict(color='red', size=10)
        ))
    except Exception:
        pass

    # SHEAR STRESS
    mask_s = d['Strain'].notna() & d['Shear Stress'].notna() & (d['Shear Stress'] > 0)
    fig.add_trace(go.Scatter(
        x=d['Strain'][mask_s],
        y=d['Shear Stress'][mask_s],
        mode='markers+lines',
        name='Shear Stress'
    ))

    # TAU_F point
    try:
        tau_f_id, tau_f_val = tau_f(d, 10)
        fig.add_trace(go.Scatter(
            x=[d["Strain"].iloc[tau_f_id]],
            y=[tau_f_val],
            mode='markers',
            name='tau_f',
            marker=dict(color='orange', size=10)
        ))
    except Exception:
        pass

    fig.update_layout(
        title=title,
        xaxis_title='Strain (%)',
        yaxis_title='Pa',
        xaxis_type='log',
        yaxis_type='log'
    )
    fig.show()

In [12]:
# read an excel file and extract the sweep and flow sheets (case-insensitive sheet names)
def read_rheometry_file(filepath):
    xls = pd.ExcelFile(filepath)
    # map lowercase stripped sheet names to actual sheet names
    sheets_map = {s.strip().lower(): s for s in xls.sheet_names}
    sweep_name = sheets_map.get('sweep')
    flow_name = sheets_map.get('flow')
    if sweep_name is None or flow_name is None:
        raise ValueError(f"Expected sheets named 'Sweep' and 'Flow' (any capitalization). Found: {xls.sheet_names}")
    sweep_df = pd.read_excel(xls, sheet_name=sweep_name)
    flow_df = pd.read_excel(xls, sheet_name=flow_name)
    return sweep_df, flow_df

# Extract rows 26 to  73 inclusive. use the two rows after 28 as column titles
def extract_relevant_data_sweep(df):
    extracted_df = df.iloc[26:72].reset_index(drop=True)
    # create new column names by combining the two rows after 28
    new_columns = []
    for col in extracted_df.columns:
        new_col_name = f"{extracted_df[col].iloc[0]} {extracted_df[col].iloc[1]}".strip()
        new_columns.append(new_col_name)
    extracted_df.columns = new_columns
    # drop the first two rows used for column names
    extracted_df = extracted_df.iloc[2:].reset_index(drop=True)
    return extracted_df

# Extract rows 18 to  60 inclusive. use the two rows after 28 as column titles

def extract_relevant_data_flow(df):
    extracted_df = df.iloc[16:59].reset_index(drop=True)
    # create new column names by combining the two rows after 18
    new_columns = []
    for col in extracted_df.columns:
        new_col_name = f"{extracted_df[col].iloc[0]} {extracted_df[col].iloc[1]}".strip()
        new_columns.append(new_col_name)
    extracted_df.columns = new_columns
    # drop the first two rows used for column names
    extracted_df = extracted_df.iloc[2:].reset_index(drop=True)
    return extracted_df

## Testing on a single measurement

In [13]:
path = r'data\rheometry\AI_11_1.xlsx'

sweep_df, flow_df = read_rheometry_file(path)
sweep_extracted = extract_relevant_data_sweep(sweep_df)
flow_extracted = extract_relevant_data_flow(flow_df)

In [14]:
sweep_extracted

,Meas. Pts. nan,Strain [%],Shear Stress [Pa],Storage Modulus [Pa],Loss Modulus [Pa],Damping Factor [1],Deflection Angle [mrad],Torque [µNm],Status []
0,1,0.01,0.21,1810,1060,0.587,0.00401,0.641,"WMa,DSO"
1,2,0.0124,0.256,1770,1050,0.591,0.00498,0.785,"WMa,DSO"
2,3,0.0155,0.32,1770,1060,0.596,0.00622,0.979,"WMa,DSO"
3,4,0.0195,0.405,1780,1070,0.603,0.00782,1.24,"WMa,DSO"
4,5,0.0246,0.511,1780,1070,0.602,0.00984,1.56,"WMa,DSO"
5,6,0.0309,0.645,1790,1070,0.602,0.0124,1.97,"WMa,DSO"
6,7,0.039,0.813,1790,1080,0.605,0.0156,2.49,"WMa,DSO"
7,8,0.0491,1.03,1790,1080,0.604,0.0196,3.14,"WMa,DSO"
8,9,0.0618,1.29,1790,1080,0.605,0.0247,3.95,"WMa,DSO"
9,10,0.0777,1.63,1790,1080,0.606,0.0311,4.98,"WMa,DSO"


In [15]:
sweep_df

,Data Series Information,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Name:,NaN,NaN,AI4_11_1 1,NaN,NaN,NaN,NaN,NaN
1,Number of Intervals:,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,Application:,NaN,NaN,RHEOPLUS/32 V3.61 21005659-33025,NaN,NaN,NaN,NaN,NaN
3,Device:,NaN,NaN,"MCR302 SN80983514; FW3.63; Slot(2,-1)",NaN,NaN,NaN,NaN,NaN
4,Measuring Date/Time:,NaN,NaN,12-07-2019; 12:47,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
103,Time Setting:,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Meas. Pts.,Strain,Shear Stress,Storage Modulus,Loss Modulus,Damping Factor,Deflection Angle,Torque,Status
106,NaN,[%],[Pa],[Pa],[Pa],[1],[mrad],[µNm],[]


# Processing all data

In [ ]:
# Load and split all into sweeps and flows

data_folder = "data/"

all_files = os.listdir(data_folder + "rheometry/")
all_files.sort()
sweeps_dataframes = []
flows_dataframes = []
successful_files = []
for filename in all_files[:]:
    try:
        sweep_df, flow_df = read_rheometry_file(data_folder + "rheometry/" + filename)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        continue
    successful_files.append(filename)
    sweep_extracted = extract_relevant_data_sweep(sweep_df)
    flow_extracted = extract_relevant_data_flow(flow_df)
    sweeps_dataframes.append(sweep_extracted)
    flows_dataframes.append(flow_extracted)

# plot all sweeps with plotly
for i, sweep_df in enumerate(sweeps_dataframes):
    plot_rheometry_plotly(
        sweep_df, title="Rheometry Data for Sample " + successful_files[i]
    )

[32]


[31]


[31]


[31]


[30]


...

[16]


[32]


[16]


[32]


[31]


## Collecting to a dataframe and saving as xlsx

In [17]:
# storage_modulus_at_eq, tau_f, tau_y points on a dataframe
def rheometry_parameters(sweep_df):
    d = _prep_sweep(sweep_df)

    storage_modulus_eq = storage_modulus_at_eq(d)
    tau_f_id, tau_f_val = tau_f(d, 10)
    tau_y_id, tau_y_val = tau_y(d, 0.02, 15)

    return {
        "storage_modulus_at_eq": storage_modulus_eq,
        "tau_f": (tau_f_id, tau_f_val),
        "tau_y": (tau_y_id, tau_y_val)
    }

rheometry = pd.DataFrame(columns=["Sample", "Storage Modulus at EQ", "Tau_f", "Tau_y"])

rheometry_list = []
for i, sweep_df in enumerate(sweeps_dataframes):
    params = rheometry_parameters(sweep_df)
    rheometry_list.append(
        {
            "Sample": successful_files[i],
            "Storage Modulus at EQ": params["storage_modulus_at_eq"],
            "Tau_f": params["tau_f"][1],
            "Tau_y": params["tau_y"][1],
        }
    )

rheometry = pd.concat([rheometry, pd.DataFrame(rheometry_list)], ignore_index=True)
rheometry.sort_values(by="Sample", inplace=True)
rheometry.to_excel(data_folder + "rheometry_parameters.xlsx", index=False)
rheometry

[32]
[31]
[31]
[31]
[30]
[32]
[33]
[26]
[26]
[33]
[32]
[32]
[16]
[18]
[30]
[32]
[32]
[16]
[17]
[17]
[16]
[32]
[16]
[32]
[31]


C:\Users\antre\AppData\Local\Temp\ipykernel_25412\3359986497.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Sample,Storage Modulus at EQ,Tau_f,Tau_y
0,AI_10_1.xlsx,145.00,30.700000,30.70000
1,AI_11_1.xlsx,1810.00,923.000000,227.00000
2,AI_11_2.xlsx,1760.00,1120.000000,248.00000
3,AI_11_3.xlsx,1570.00,858.000000,211.00000
4,AI_11_4.xlsx,1590.00,869.000000,171.00000
5,AI_11_5.xlsx,1680.00,884.000000,266.00000
6,AI_12_1.xlsx,305.00,151.000000,81.50000
7,AI_13_1.xlsx,284.00,0.047900,18.10000
8,AI_14_1.xlsx,11100.00,2790.000000,410.00000
9,AI_15_1.xlsx,577.00,503.000000,128.00000


In [18]:
# From flow data, extract viscosity at 10 1/s for all samples 
flow_viscosities_at_10 = []
for i, flow_df in enumerate(flows_dataframes):
    flow_df_numeric = flow_df.apply(pd.to_numeric, errors='coerce')
    mask = flow_df_numeric['Shear Rate [1/s]'] == 10
    viscosity_values = flow_df_numeric.loc[mask, 'Viscosity [Pa·s]']
    if not viscosity_values.empty:
        viscosity_at_10 = viscosity_values.iloc[0]
    else:
        viscosity_at_10 = np.nan  # or some default value if 10 1/s not found
    flow_viscosities_at_10.append(viscosity_at_10)

# print flows at 10 and name of file
for i, viscosity in enumerate(flow_viscosities_at_10):
    print(f"{successful_files[i]}: Viscosity at 10 1/s = {viscosity} Pa·s")

# append viscosity to rheometry dataframe
rheometry['Viscosity at 10 1/s (Pa·s)'] = flow_viscosities_at_10
rheometry.to_excel(data_folder + "rheometry_parameters.xlsx", index=False)

AI_10_1.xlsx: Viscosity at 10 1/s = 28.3 Pa·s
AI_11_1.xlsx: Viscosity at 10 1/s = 147.0 Pa·s
AI_11_2.xlsx: Viscosity at 10 1/s = 140.0 Pa·s
AI_11_3.xlsx: Viscosity at 10 1/s = 137.0 Pa·s
AI_11_4.xlsx: Viscosity at 10 1/s = 137.0 Pa·s
AI_11_5.xlsx: Viscosity at 10 1/s = 138.0 Pa·s
AI_12_1.xlsx: Viscosity at 10 1/s = 48.9 Pa·s
AI_13_1.xlsx: Viscosity at 10 1/s = 115.0 Pa·s
AI_14_1.xlsx: Viscosity at 10 1/s = 483.0 Pa·s
AI_15_1.xlsx: Viscosity at 10 1/s = 70.7 Pa·s
AI_16_1.xlsx: Viscosity at 10 1/s = 54.0 Pa·s
AI_17_1.xlsx: Viscosity at 10 1/s = 66.0 Pa·s
AI_18_1.xlsx: Viscosity at 10 1/s = 2210 Pa·s
AI_19_1.xlsx: Viscosity at 10 1/s = 2.01 Pa·s
AI_1_1.xlsx: Viscosity at 10 1/s = nan Pa·s
AI_1_2 second run.xlsx: Viscosity at 10 1/s = 13.0 Pa·s
AI_1_2.xlsx: Viscosity at 10 1/s = 13.2 Pa·s
AI_2_1.xlsx: Viscosity at 10 1/s = 0.179 Pa·s
AI_3_1.xlsx: Viscosity at 10 1/s = 4.88 Pa·s
AI_4_1.xlsx: Viscosity at 10 1/s = 0.192 Pa·s
AI_5_1.xlsx: Viscosity at 10 1/s = 0.011 Pa·s
AI_6_1.xlsx: Viscosit